In [2]:
!pip install geopandas
!apt install libspatialindex-dev
!pip install rtree
!pip install pygeos
import geopandas as gpd
import json
import pandas as pd
import rtree
import pygeos

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
shapefile = gpd.read_file("/content/other_climate_2007_koppen_geiger.shp", encoding="utf-8")

with open('/content/drive/MyDrive/mapillary/train.json', encoding="UTF-8") as f:
  data = json.load(f)

coords = pd.DataFrame.from_dict(data, orient="index")
coords = coords.drop("focal", axis=1)
coords = coords.drop("make", axis=1)
coords = coords.drop("model", axis=1)
coords = coords.rename(columns={"lat": "Latitude", "lon": "Longitude"})

In [20]:
df = gpd.GeoDataFrame(
    coords, geometry=gpd.points_from_xy(coords.Longitude, coords.Latitude))
df = df.set_crs(shapefile.crs)
newpd = gpd.sjoin_nearest(df, shapefile, distance_col='distances', max_distance=1)


/usr/local/lib/python3.7/dist-packages/geopandas/array.py:348: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  stacklevel=stacklevel,


In [21]:
newpd.sort_values(by=['distances'],ascending=False,inplace=True)
newpd

,Latitude,Longitude,geometry,index_right,identity,climate,distances
yYjKcJlC1aneLqPeOdXEdQ,-20.974643,55.656139,POINT (55.65614 -20.97464),1723,2eca8613-09f6-d8b4-cf36-ffb0ad244257,Af Tropical-Rainforest,6.751366
h7LxUZy4JwSXFRGnjJNqGg,-20.974326,55.655995,POINT (55.65599 -20.97433),1723,2eca8613-09f6-d8b4-cf36-ffb0ad244257,Af Tropical-Rainforest,6.751136
rtYgCIXfKmvOmTt-JzyBiw,-21.202010,55.574361,POINT (55.57436 -21.20201),1723,2eca8613-09f6-d8b4-cf36-ffb0ad244257,Af Tropical-Rainforest,6.743985
oAb0JYQF3m6ruZiJDIFFhw,-21.202010,55.574361,POINT (55.57436 -21.20201),1723,2eca8613-09f6-d8b4-cf36-ffb0ad244257,Af Tropical-Rainforest,6.743985
nJ8JPlscDspfPoakyFuUTQ,-21.201896,55.574356,POINT (55.57436 -21.20190),1723,2eca8613-09f6-d8b4-cf36-ffb0ad244257,Af Tropical-Rainforest,6.743942
...,...,...,...,...,...,...,...
Y3GExP4py5SG3b2ExT9d-w,34.054464,-4.975065,POINT (-4.97507 34.05446),3944,fa65bcb7-9e3e-bcc9-f69c-f0de974f4919,Csa Temperate-Dry_Summer-Hot_Summer,0.000000
Y-iAM8jOzXpFbNAy8oyWAQ,35.571125,-5.376278,POINT (-5.37628 35.57112),3944,fa65bcb7-9e3e-bcc9-f69c-f0de974f4919,Csa Temperate-Dry_Summer-Hot_Summer,0.000000
ZLGepHO2WhsHKUcLf_XrzA,33.552514,-7.592440,POINT (-7.59244 33.55251),3944,fa65bcb7-9e3e-bcc9-f69c-f0de974f4919,Csa Temperate-Dry_Summer-Hot_Summer,0.000000
X_2DUatguGHXy-_grCbSXg,34.036040,-5.022423,POINT (-5.02242 34.03604),3944,fa65bcb7-9e3e-bcc9-f69c-f0de974f4919,Csa Temperate-Dry_Summer-Hot_Summer,0.000000


In [11]:
newpd['climate'].value_counts()

Cfb Temperate-Withouth_dry_season-Warm_Summer    130063
Cfa Temperate-Withouth_dry_season-Hot_Summer     108767
Dfb Cold-Withouth_dry_season-Warm_Summer          92807
Aw Tropical-Savanna                               50732
Csa Temperate-Dry_Summer-Hot_Summer               47288
BSk Arid-Steppe-Cold                              41605
Csb Temperate-Dry_Summer-Warm_Summer              34641
BWh Arid-Desert-Hot                               29194
Dfa Cold-Withouth_dry_season-Hot_Summer           29067
BSh Arid-Steppe-Hot                               18333
Am Tropical-Monsoon                               15057
Af Tropical-Rainforest                            13246
Cwa Temperate-Dry_Winter-Hot_Summer               10308
BWk Arid-Desert-Cold                               9852
Cwb Temperate-Dry_Winter-Warm_Summer               9573
Dfc Cold-Withouth_dry_season-Cold_Summer           7655
Dsb Cold-Dry_Summer-Warm_Summer                    3895
Cwc Temperate-Dry_Winter-Cold_Summer            

In [19]:
newpd.to_csv('climates.csv')